# Starting and testing OVRO-LWA

### Setup

1. Use dashboard to confirm that services are up for etcd, bifrost pipelines, and data capture.
2. Identify server names for snap2, gpu, and data capture.
3. Set list of server names in notebook.
4. Execute cells


### F-eng
* Repo https://github.com/realtimeradio/caltech-lwa
* python install "control_sw"

### X-eng
* Repo https://github.com/realtimeradio/caltech-bifrost-dsp
* python install "pipeline-control"

### mnc
* Repo https://github.com/ovro-lwa/mnc_python
* python install "mnc_python"

In [1]:
snap2names = [f'snap{i:02}' for i in range(1, 12)]  # f-eng
feng_config = '/home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml'  # f-eng and x-eng
xhosts = ['lxdlwagpu02']  # x-eng
npipeline = 4  # x-eng
drips = ['10.41.0.25', '10.41.0.41']  # data recorder (two ips for lxdlwagpu09)
dest_ip = list(sorted(drips*2))  # correlator -> data recorder (destination IPs)
dest_port = [10001+i//npipeline for i in range(npipeline*len(xhosts))]  # correlator -> data recorder (destination ports)
drroot = [f'drvs{ip.split(".")[-1]}' for ip in drips]  # data recorder root names
drids = [f'{drr}{dp-10000:02}' for dp in set(dest_port) for drr in drroot]  # data recorder ids

In [2]:
print(snap2names, xhosts, drips, dest_ip, dest_port, drids)

['snap01'] ['lxdlwagpu02'] ['10.41.0.25', '10.41.0.41'] ['10.41.0.25', '10.41.0.25', '10.41.0.41', '10.41.0.41'] [10001, 10001, 10001, 10001] ['drvs2501', 'drvs4101']


In [3]:
from lwa_f import snap2_fengine
from lwa352_pipeline_control import Lwa352CorrelatorControl
from mnc import mcs, common, ezdr

In [4]:
for snap2name in snap2names:
    lwa_f = snap2_fengine.Snap2Fengine(snap2name)
#    lwa_f.program()
    lwa_f.cold_start_from_config(feng_config)

2022-04-15 23:05:24,106 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-04-15 23:05:24,327 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-04-15 23:05:24,539 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2022-04-15 23:05:24,550 - lwa_f.snap2_fengine:snap01 - INFO - Trying to configure output with config file /home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml
2022-04-15 23:05:24,553 - lwa_f.snap2_fengine:snap01 - INFO - Loading existing firmware to snap01
File in flash is:  snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg
2022-04-15 23:05:45,709 - lwa_f.snap2_fengine:snap01 - INFO - Loaded snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg
2022-04-15 23:05:46,024 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-04-15 23:05:46,259 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-04-15 23:06:06,672 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane delays:
[[304, 296, 312, 296, 296, 296, 296, 296], [272, 264, 264, 256, 264, 264, 256, 256], [248, 248, 248, 248, 248, 248, 248, 256], [280, 296, 280, 280, 288, 280, 288, 280], [256, 256, 256, 264, 264, 256, 256, 256], [240, 240, 240, 240, 232, 232, 232, 232], [256, 232, 232, 240, 240, 248, 240, 240], [280, 280, 280, 280, 272, 288, 288, 272]]
2022-04-15 23:06:06,672 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane slacks:
[[40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [32, 40, 40, 40, 32, 32, 32, 32], [40, 40, 40, 40, 40, 40, 40, 40], [32, 40, 40, 40, 40, 32, 40, 32], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 32, 40, 40, 32, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40]]
2022-04-15 23:06:06,673 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|-----XXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,674 - lwa_f.blocks.block:snap01 - INF

2022-04-15 23:06:06,704 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,704 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,705 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,705 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,706 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,706 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:06:06,707 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 7, Lane 0:    XXXXXXXX

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


...

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-04-15 23:07:07,587 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 0 because nowhere was good
2022-04-15 23:07:07,600 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 1 because nowhere was good
2022-04-15 23:07:07,613 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 2 because nowhere was good
2022-04-15 23:07:07,626 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 3 because nowhere was good
2022-04-15 23:07:08,679 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 Scanning data delays


100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-04-15 23:07:15,995 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane delays:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [280, 288, 280, 296, 288, 288, 280, 288], [224, 224, 216, 216, 224, 216, 232, 216], [272, 256, 264, 272, 264, 264, 264, 264], [304, 304, 304, 296, 296, 296, 296, 296]]
2022-04-15 23:07:15,996 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane slacks:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [40, 40, 40, 40, 40, 40, 32, 40], [40, 40, 32, 32, 40, 32, 40, 32], [32, 32, 32, 32, 32, 40, 32, 32], [40, 40, 32, 40, 32, 32, 32, 32]]
2022-04-15 23:07:15,996 - lwa_f.blocks.block:snap01 - WARNING - adc - Delay solutions have small slack
2022-04-15 23:07:15,997 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:15,998 - lwa_f.blocks.block:snap0

2022-04-15 23:07:16,028 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 1:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:16,029 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:16,029 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:16,030 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:16,031 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:16,032 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-04-15 23:07:16,033 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXX

In [5]:
lwa_corr = Lwa352CorrelatorControl(xhosts, npipeline_per_host=npipeline)
lwa_corr.stop_pipelines()   # maybe not needed?
lwa_corr.start_pipelines() 
lwa_corr.configure_corr(dest_ip=dest_ip, dest_port=dest_port)

2022-04-15 23:08:26 [INFO    ] Pipelines all appear to be ready after 17.1 seconds
2022-04-15 23:08:26 [INFO    ] Setting max data output rate per pipeline to 5000.0 Mbit/s
2022-04-15 23:08:26 [INFO    ] Setting pipeline lxdlwagpu02:0 data destination to 10.41.0.25:10001
2022-04-15 23:08:26 [INFO    ] Setting pipeline lxdlwagpu02:1 data destination to 10.41.0.25:10001
2022-04-15 23:08:26 [INFO    ] Setting pipeline lxdlwagpu02:2 data destination to 10.41.0.41:10001
2022-04-15 23:08:26 [INFO    ] Setting pipeline lxdlwagpu02:3 data destination to 10.41.0.41:10001
2022-04-15 23:08:26 [INFO    ] Arming correlator core


KeyError: 'sync_time'

In [6]:
rs = ezdr.Lwa352RecorderControl('slow')
rs.start()

Found 0 slow recorders: 


[]

### Read data and visualize/test

## Stop

In [9]:
rs.stop()
lwa_corr.stop_pipelines()